In [3]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings
warnings.filterwarnings('ignore')


In [15]:
# parameters
customer_id = '5'
formatted_attribute = 'Cord Length'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Massage & Manipulation Tools"""

regex_pattern_inch=r'(?i)(cord length measures\s?\d+.?\d*.?\")|(cord measures\s?\d+.?\d*.?\")|()'
regex_pattern_foot=r"(?i)((cord length measures\s?\d+.?\d*.?ft))|((cord length measures\s?\d+.?\d*.?\'))|((cord measures\s?\d+.?\d*.?ft))|((cord measures\s?\d+.?\d*.?\'))|()"

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [16]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [25]:
df = query_from_file(file_name='query/uncurated.sql', params=params)
df.head(1)

product_id external_id  \
0   817192885    63228342   

                                            product_name  \
0  Simple Designs Porcelain Wise Owl Table Lamp in White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         long_desc  \
0  Add a gentle glow of light during nighttime with the Simple Designs Porcelain Wise Owl Table Lamp. Made from porcelain, it is adorably shaped like your favorite animal, blending with the decor while adding a soothing touch. Features adorable owl design Provides a soft light that can be used as a table lamp or night light On/off switch on base Plastic/iron Wipe clean Uses one 25-watt bulb (not included) Plugs into any standard AC outlet Measures 6.89” L x 7.48” W x 9.84” H Cord measures 6' L 1-year limited manufacturer's warranty Imported   

   customer_id attributes buckets values  
0            5       None   Lamps   None

In [30]:
df[df['matches'].astype(str)=='720']

Empty DataFrame
Columns: [product_id, external_id, product_name, long_desc, customer_id, attributes, buckets, values, matches]
Index: []

In [ ]:
error

In [18]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [19]:
def clean_data(data):
    data['matches']=data['matches'].str.get(0)
#     data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())

In [26]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_inch, x))
inch=df[df['matches'].astype(str)!='[]']
clean_data(inch) #might throw an error and have to drop nane
inch['matches']=inch['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'\\",'',str(measure))).replace("'", '').apply(lambda quote: quote.replace('"', ''))
inch

product_id external_id  \
8      817188890    62679770   
9      817477059    62679626   
10     817188889    62679619   
12     817477060    62679763   
20     988799831    62944199   
...          ...         ...   
7121   817358891    67572694   
7132   817359224    77525027   
7133   817269973    62970815   
7135   817543356    63625905   
7137   817293239    64180182   

                                                                       product_name  \
8               Gallery 10-Light Ceiling Mount Swarovski Crystal Trimmed Chandelier   
9                Gallery 5-Light Ceiling Mount Swarovski Crystal Trimmed Chandelier   
10                       Gallery 5-Light Ceiling Mount Crystal Swag Plug Chandelier   
12                      Gallery 10-Light Ceiling Mount 40MM Crystal Ball Chandelier   
20           Authentic Eclectic Terry 4-Light Pendant Chandelier in Polished Nickel   
...                                                                             ...   
7121                             JONATHAN Y Simon Pinecone Pendant Light in Natural   
7132                                  ED Ellen DeGeneres Ferrelli Floor Lamp in Oak   
7133  Southern Enterprises Redden Corner Convertible Electric Fireplace in Espresso   
7135                             Adesso Piper LED Table Lamp in Black/Antique Brass   
7137      ELK Lighting Riley 1-Light Pendant in Satin Nickel with Clear Glass Shade   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   long_desc  \
8                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

# Foot

In [27]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_foot, x))
foot=df[df['matches'].astype(str)!='[]']
clean_data(foot) #might throw an error and have to drop nane
foot['matches']=foot['matches'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\\|\'",'',str(measure))).replace("'", '').apply(lambda quote: quote.replace('"', ''))
foot['matches']=foot['matches'].astype(float).apply(lambda x: float(x*12)).astype(int)
foot

product_id external_id  \
0      817192885    63228342   
17     817188037    62520386   
18     817188036    62520263   
19     817187879    62493604   
20     988799831    62944199   
...          ...         ...   
7122   988387105    65658949   
7124   988522177    61130357   
7134   817521564    61094574   
7142   817277443    62236225   
7145   988474635    69589180   

                                                                   product_name  \
0                         Simple Designs Porcelain Wise Owl Table Lamp in White   
17    Décor Therapy Sea Horse Accent Lamp in Silver Leaf with White Linen Shade   
18               Décor Therapy Walter Floor Lamp in Dark Wood with Fabric Shade   
19                                  Large Brushed Steel Lamp Base with CFL Bulb   
20       Authentic Eclectic Terry 4-Light Pendant Chandelier in Polished Nickel   
...                                                                         ...   
7122                                                   Rope Table Lamp in White   
7124                                     Jimco Ceramic Table Lamp in Blue/White   
7134                                        Vanderbilt CFL Floor Lamp in Nickel   
7142                                           Surya Croyle Table Lamp in Ivory   
7145                                                 Niklas Table Lamp in White   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               long_desc  \
0                        Add a gentle glow of light during nighttime with the Simple Designs Porcelain Wise Owl Table Lamp. Made from porcelain, it is adorably shaped like your favorite animal, blending with the decor while adding a soothing touch. Features adorable owl design Provides a soft light that can be used as a table lamp or night light On/off switch on base Plastic/iron Wipe clean Uses one 25-watt bulb (not included) Plugs into any standard AC outlet Measures 6.89” L x 7.48” W x 9.84” H Cord measures 6' L 1-year limited manufacturer's warranty Imported   
17        Add a costal touch to your decor with the Sea Horse Accent Lamp by Decor Therapy. This eye-catching lamp is constructed of resin and features a beautifully detailed sea horse perched on top of a square base and is completed with a round white linen shade. Features a beautifully detailed sea horse perched on top of a square base Costal style On/off switch on neck Round white linen shade Silver leaf finish Resin base Uses one 40-watt bulb (not included) Plugs into any standard AC outlet Measures 16\" H Lampshade measures 7\" H Cord measures 8' L Imported   
18                     Bring a simple yet refined look to your home décor with the Walter Floor Lamp from Décor Therapy. Finished in dark wood color, this piece boasts a sleek silhouette plus a cream bell shade to exude attractive functionality in your room of choice. Features a sleek silhouette This lamp will transition well from room to room 3-way switch Cream bell shade Dark wood finish Resin/steel base Uses one 100-watt bulb (not included) Plugs into any standard AC outlet Measures 62\" H Lampshade measures 14\" diameter x  11\" H Cord measures 8' L Imported   
19                                                                                                                                             Transform the look and feel of your space with the Large Brushed Steel Lamp Base. Sporting a sleek brushed stainless steel finish, this chic lamp base pairs well with a variety of shades for a 

In [31]:
foot[foot['matches'].astype(str)=='720']

product_id external_id  \
467    817415606    69512970   
531    817603659    68128098   
1183   817415569    69512898   
1763   817635321    69512902   
2157   988752076    69529596   
3576   817635314    69512886   
3645   988752179    69529572   
3867   817635352    69512891   
3874   817369724    68128074   
3949  1176138270    69512957   
4157   817415566    69512889   
4302   817415630    69512968   
4329   817369725    68128104   
4634   988752072    69529461   
4698   817415627    69512892   
4896   817603660    68128081   
5691   988749170    69512899   
6143   817635315    69512887   
6181   817635318    69512896   
6428   817635353    69512890   
6502   817415605    69512969   
6917   817635320    69512901   

                                                          product_name  \
467                       Safavieh Nilla Table Lamp in Navy (Set of 2)   
531    Elegant Designs Modern Leather Table Lamp With USB Port in Teal   
1183                               Safavieh Jacoby Table Lamp in Cream   
1763                                         Safavieh Velma Table Lamp   
2157                     StyleCraft Crystal Table Lamp in Black/Nickel   
3576                          Safavieh Deco Leather Table Lamp in Navy   
3645                       StyleCraft Newnan Steel Table Lamp in Black   
3867                              Safavieh Mercurio Table Lamp in Blue   
3874  Elegant Designs Modern Leather Table Lamp With USB Port in Black   
3949                    Safavieh Kaden Table Lamps in Brass (Set of 2)   
4157                                Safavieh Lola Column Lamp in White   
4302               Safavieh Karlen Table Lamp in Black/Gold (Set of 2)   
4329  Elegant Designs Modern Leather Table Lamp With USB Port in White   
4634                             StyleCraft Allen Table Lamp in Bronze   
4698                      Safavieh Mercurio Table Lamp in Ivory/Silver   
4896   Elegant Designs Modern Leather Table Lamp With USB Port in Grey   
5691                     Safavieh Greek Key Table Lamp in Antique Gold   
6143                        Safavieh Shelley Gourd Table Lamp in White   
6181                               Safavieh Estill Table Lamp in Black   
6428                             Safavieh Mercurio Table Lamp in Clear   
6502                     Safavieh Nilla Table Lamp in Cream (Set of 2)   
6917         Safavieh Haley Double Sphere Table Lamp in Antique Silver   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            long_desc  \
467                                                                                                           Enhance your space with the modern look of the Safavieh Nilla Table Lamp. Illuminating sleek glass curves, the acrylic base and empire shade on this lamp makes it the perfect accessory for updating any room decor. Bring modern magic to any contemporary interior with this stunning genie-style table lamp Rotary on/off switch Empire shade Glass construction Wipe clean Uses two 9-watt LED bulbs (included) Plugs into a standard outlet Measures 15\" L x 15\" W x 26.5\" H Cord measures 60' L Weighs 9.9 lb. Energy Star certified Imported   
531                                                       Adorn your end table in functional style with the Elegant Designs Modern Leather Table Lamp with USB Port. Boasting an embossed leather base, this sleek and modern lamp is complete with a convenient USB port to

In [28]:
inch['matches'].explode().value_counts()

72    224
96    211
60    125
6      89
84     59
     ... 
82      1
42      1
5       1
37      1
38      1
Name: matches, Length: 98, dtype: int64

In [29]:
foot['matches'].explode().value_counts()

72     439
96     395
144     63
60      44
192     32
84      28
180     24
720     22
120     17
48      10
108      7
168      3
36       3
864      3
240      3
252      3
300      3
360      3
816      2
12       2
135      1
196      1
228      1
73       1
70       1
384      1
420      1
30       1
24       1
Name: matches, dtype: int64

In [14]:
matches=pd.concat([inch,foot])
matches

product_id external_id                              product_name  \
85   817557218    64875163  Viante® 1.5 Liter Digital Steeper Kettle   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            long_desc  \
85  Boil and brew your favorite drinks with the Café Vianté Digital Steeper Kettle. This electric kettle features presets for your favorite teas or French press coffee and a built-in removable tea steeping filter so you can maximize tea time. Built-in tea steeping filter steeps your tea while the kettle is boiling the water, which means less waiting for that perfect cup Touch operated digital temperature presets for your favorite teas and French press coffee Keep Warm setting ensures your beverage stays warm while you enjoy each cup Glass Hand wash Plug into a standard outlet 1.5 liter capacity Measures 9.45\" L x 7.5\" W x 8.85\" H Cord measures 31\" L 1-year manufacturer's warranty Imported KET-100   

    customer_id attributes               buckets values matches  
85            5       None  Teapots & Teakettles   None      31

In [ ]:
error

In [ ]:
x=pd.DataFrame()
x['col']=[1.00, 1.10, 1.02, 1.003]
x['col']=x['col'].astype(str).apply(lambda quote: quote.replace('.0', ''))
x

In [ ]:
matches[curation_col] = matches['matches'].astype(str)+' in'
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)